## Libraries

In [1]:
import os
import glob
import random
import librosa
import librosa.display

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.layers import Conv2D, MaxPooling2D, RNN, LSTM, GRU, Flatten, Dropout, ReLU, BatchNormalization, Dense
from tensorflow.keras.activations import tanh, sigmoid, softmax, relu
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint



## Preprocess

In [2]:
# Define constants
sample_rate = 16000
Samples = 16000
Dataset = "Dataset"

In [3]:
file_names = ['bed','bird','cat','dog','down', 'eight']

In [4]:
# folder_path = [os.path.join(Dataset.na) for name in file_names]

In [5]:
# print(folder_path)

In [6]:
def load_audio(path):
    audio, _ = librosa.load(path, sr=sample_rate)
    if len(audio) < Samples:
        audio = np.pad(audio, (0, Samples - len(audio)))  # Fixed padding
    else:
        audio = audio[:Samples]
    return audio

## MEL-Spectrum

In [7]:
# sample_rate = 16000

In [8]:
def get_mel(audio):
    mel_spec = librosa.feature.melspectrogram(y=audio, sr=sample_rate)
    mel_spec = librosa.power_to_db(mel_spec, ref=np.max)  
    return mel_spec

In [9]:
# x,y,label_map = load_data()

In [10]:
import numpy
numpy.version.version

'1.25.2'

In [11]:

def load_data():
    x, y = [], []
    label_map = {label: i for i, label in enumerate(file_names)}

    for label in file_names:
        files = glob.glob(os.path.join(Dataset, label, "*.wav"))
        print(" --" * 5)
        print("Sample Files 1:")
        print(" --" * 5)

        for file in files:
            audio = load_audio(file)  # Fixed variable name
            mel_spec = get_mel(audio)
            x.append(mel_spec)
            y.append(label_map[label])
    
    return np.array(x), np.array(y), label_map

# Load data
x, y, label_map = load_data()


 -- -- -- -- --
Sample Files 1:
 -- -- -- -- --
 -- -- -- -- --
Sample Files 1:
 -- -- -- -- --
 -- -- -- -- --
Sample Files 1:
 -- -- -- -- --
 -- -- -- -- --
Sample Files 1:
 -- -- -- -- --
 -- -- -- -- --
Sample Files 1:
 -- -- -- -- --
 -- -- -- -- --
Sample Files 1:
 -- -- -- -- --


## Train Test Split

In [12]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=32)


In [13]:
print(f"train sample : {len(x_train)}, {len(y_train)}")


train sample : 8910, 8910


In [14]:
x_train = x_train[..., np.newaxis]

## Model

In [15]:
def build_model(input_shape, num_classes):
    model = Sequential([

        Conv2D(32,(3,3), activation = "relu", input_shape = input_shape),
        MaxPooling2D((2,2)),
        Conv2D(64,(3,3),activation = "relu"),
        MaxPooling2D((2,2)),
        Conv2D(128,(3,3),activation = "relu"),
        MaxPooling2D((2,2)),
        Flatten(),
        Dense(128, activation = "relu"),
        Dropout(0.2),
        Dense(num_classes, activation = "softmax")
     ])
    return model

In [16]:
model = build_model(x_train[0].shape, len(file_names))

In [17]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 126, 30, 32)       320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 63, 15, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 61, 13, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 30, 6, 64)         0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 28, 4, 128)        73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 14, 2, 128)        0

In [18]:
model.compile(optimizer="adam", loss = "sparse_categorical_crossentropy", metrics = ['accuracy'])

In [20]:
early = EarlyStopping(monitor = "val_loss", patience = 5, restore_best_weights = True)

In [21]:
epoch= 20
batch_size = 32

history = model.fit(x_train, y_train, validation_data = (x_test,y_test), epochs = epoch, batch_size = batch_size, callbacks= [early])

Epoch 1/20
279/279 [==============================] - 45s 160ms/step - loss: 0.1394 - accuracy: 0.9510 - val_loss: 0.2295 - val_accuracy: 0.9174
Epoch 2/20
279/279 [==============================] - 44s 159ms/step - loss: 0.1252 - accuracy: 0.9574 - val_loss: 0.1653 - val_accuracy: 0.9452
Epoch 3/20
279/279 [==============================] - 42s 151ms/step - loss: 0.1112 - accuracy: 0.9612 - val_loss: 0.1814 - val_accuracy: 0.9425
Epoch 4/20
279/279 [==============================] - 44s 158ms/step - loss: 0.0952 - accuracy: 0.9664 - val_loss: 0.1982 - val_accuracy: 0.9408
Epoch 5/20
279/279 [==============================] - 43s 156ms/step - loss: 0.0804 - accuracy: 0.9728 - val_loss: 0.2657 - val_accuracy: 0.9228
Epoch 6/20
279/279 [==============================] - 43s 155ms/step - loss: 0.0966 - accuracy: 0.9632 - val_loss: 0.2074 - val_accuracy: 0.9309
Epoch 7/20
279/279 [==============================] - 54s 193ms/step - loss: 0.0856 - accuracy: 0.9691 - val_loss: 0.1581 - val_ac


KeyboardInterrupt



In [ ]:
import matplotlib.pyplot as plt

plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
loss, accuracy = model.evaluate(x_train, y_train)
print(f"Training Accuracy: {accuracy:.2f}")


## inference:

In [ ]:
index = random.randint(0, len(x_test) - 1)
sample = x_test[index]
true_label = y_test[index]

In [ ]:
sample_input = np.expand_dims(sample, axis = 0)

In [ ]:
prediction = model.predict(sample_input)
prediction_label = np.argmax(prediction)



In [ ]:
label_mappppp = {v : k for k, v for v in label_map.itme()}
print(f"Actual Label :{label_mappppp[true_label]}")
print(f"Predicted Label :{label_mappppp[prediction_label]}")